In [1]:
info = """
LangChain 1.0 - Structured Output (结构化输出)
=============================================

本模块重点讲解：
1. 使用 Pydantic 定义输出模式
2. with_structured_output() 方法
3. 嵌套模型和复杂结构
4. 枚举类型和验证
5. 实际应用场景

⚠️ 注意：
- with_structured_output() 可能在某些模型上不完全支持
- 如遇到错误，模块会自动使用 JSON 解析作为 fallback
"""

print(info)


LangChain 1.0 - Structured Output (结构化输出)

本模块重点讲解：
1. 使用 Pydantic 定义输出模式
2. with_structured_output() 方法
3. 嵌套模型和复杂结构
4. 枚举类型和验证
5. 实际应用场景

⚠️ 注意：
- with_structured_output() 可能在某些模型上不完全支持
- 如遇到错误，模块会自动使用 JSON 解析作为 fallback



In [11]:
import os
import json
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from pydantic import BaseModel, Field
from langchain.messages import HumanMessage
from typing import List,Optional,Type,TypeVar
from enum import Enum

In [3]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )

In [4]:
# 初始化模型
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)


In [5]:
# 辅助函数
T = TypeVar("T", bound=BaseModel)


In [ ]:
def create_safe_structured_llm(output_class):
    base_llm = model.with_structured_output(output_class)
    
    class SafeStructuredLLM:
        def invoke(self,prompt):
            try:
                return base_llm.invoke(prompt)
            except Exception as e:
                print(f"  ⚠️ 结构化输出失败，使用 fallback: {e}")
                return safe_structured_output(prompt, output_class)
    return SafeStructuredLLM()


In [12]:
def safe_structured_output(prompt: str, output_class: Type[T]) -> T:
    """
    安全的结构化输出函数
    
    先尝试 with_structured_output，失败则使用 JSON 解析 fallback
    """
    # 尝试使用 with_structured_output
    try:
        structured_llm = create_safe_structured_llm(output_class)
        result = structured_llm.invoke(prompt)
        return result
    except Exception as e:
        print(f"  ⚠️ with_structured_output 失败: {e}")
        print("  📝 使用 JSON 解析 fallback...")
     # Fallback: 手动 JSON 解析
    json_prompt = f"""{prompt}

请严格按照以下JSON格式返回（不要添加任何其他内容）：
{json.dumps(output_class.model_json_schema().get('properties', {}), indent=2, ensure_ascii=False)}

只返回JSON，不要其他文字。"""
    
    response = model.invoke([HumanMessage(content=json_prompt)])
    content = response.content.strip()
    
    # 清理 Markdown 格式
    if "```json" in content:
        content = content.split("```json")[1].split("```")[0]
    elif "```" in content:
        content = content.split("```")[1].split("```")[0]
    
    try:
        data = json.loads(content.strip())
        return output_class.model_validate(data)
    except Exception as e2:
        print(f"  ❌ JSON 解析也失败: {e2}")
        raise ValueError(f"无法解析结构化输出: {e2}")

In [23]:
class Person(BaseModel):
    '''人物信息'''
    name: str = Field(description="姓名")
    age: int = Field(description="年龄")
    occupation: str = Field(description="职业")    

In [24]:
print(    """
    示例1：基础结构化输出

    使用 with_structured_output() 将 LLM 输出转为 Pydantic 对象
    """)


    示例1：基础结构化输出

    使用 with_structured_output() 将 LLM 输出转为 Pydantic 对象
    


In [25]:
print("\n" + "="*70)
print("示例 1：基础结构化输出 - Pydantic 模型")
print("="*70)


示例 1：基础结构化输出 - Pydantic 模型


In [26]:
result = safe_structured_output("张三是一名 30 岁的软件工程师", Person)
result

Person(name='张三', age=30, occupation='软件工程师')

In [27]:
print(f"\n返回类型: {type(result)}")
print(f"姓名: {result.name}")
print(f"年龄: {result.age}")
print(f"职业: {result.occupation}")


返回类型: <class '__main__.Person'>
姓名: 张三
年龄: 30
职业: 软件工程师


In [28]:
print("\n关键点：")
print("  - with_structured_output(Person) 返回 Person 对象")
print("  - 不需要手动解析 JSON")
print("  - 自动类型验证（age 必须是 int）")



关键点：
  - with_structured_output(Person) 返回 Person 对象
  - 不需要手动解析 JSON
  - 自动类型验证（age 必须是 int）


In [29]:
print( """
    示例2：提取多个对象

    从文本中提取多个结构化对象
    """)


    示例2：提取多个对象

    从文本中提取多个结构化对象
    


In [30]:
print("\n" + "="*70)
print("示例 2：提取多个对象 - 列表")
print("="*70)


示例 2：提取多个对象 - 列表


In [31]:
# 定义数据结构
class Book(BaseModel):
    '''书籍信息'''
    title : str = Field(description="书名")
    author : str = Field(description="作者")
    year : int = Field(description="出版年份")
    

In [32]:
class BookList(BaseModel):
    '''书籍列表'''
    books: List[Book] = Field(description="书籍列表")

In [33]:
structured_llm = create_safe_structured_llm(BookList)

In [34]:
text = """
《三体》是刘慈欣 2008 年的科幻小说。
《流浪地球》也是刘慈欣的作品，2000 年出版。
《北京折叠》是郝景芳 2012 年的小说。
"""


In [35]:
print(f"\n文本: {text.strip()}")
result = structured_llm.invoke(f"从以下文本中提取所有书籍信息：\n{text}")



文本: 《三体》是刘慈欣 2008 年的科幻小说。
《流浪地球》也是刘慈欣的作品，2000 年出版。
《北京折叠》是郝景芳 2012 年的小说。


In [36]:
result

BookList(books=[Book(title='三体', author='刘慈欣', year=2008), Book(title='流浪地球', author='刘慈欣', year=2000), Book(title='北京折叠', author='郝景芳', year=2012)])

In [37]:
print(f"\n提取到 {len(result.books)} 本书：")
for i, book in enumerate(result.books, 1):
    print(f"  {i}. 《{book.title}》 - {book.author} ({book.year})")



提取到 3 本书：
  1. 《三体》 - 刘慈欣 (2008)
  2. 《流浪地球》 - 刘慈欣 (2000)
  3. 《北京折叠》 - 郝景芳 (2012)


In [38]:
print("\n关键点：")
print("  - books: List[Book] 定义列表类型")
print("  - LLM 自动识别并提取多个对象")
print("  - 返回的是 Python 列表，可直接遍历")


关键点：
  - books: List[Book] 定义列表类型
  - LLM 自动识别并提取多个对象
  - 返回的是 Python 列表，可直接遍历


In [39]:
print("""
    示例3：嵌套模型

    处理复杂的层级结构
    """)


    示例3：嵌套模型

    处理复杂的层级结构
    


In [40]:
print("\n" + "="*70)
print("示例 3：嵌套模型 - 复杂结构")
print("="*70)


示例 3：嵌套模型 - 复杂结构


In [41]:
class Address(BaseModel):
    """地址"""
    city: str = Field(description="城市")
    district: str = Field(description="区")

class Company(BaseModel):
    """公司信息"""
    name: str = Field(description="公司名称")
    employee_count: int = Field(description="员工数量")
    address: Address = Field(description="公司地址")


In [42]:
structured_llm = create_safe_structured_llm(Company)

print("\n提示: 阿里巴巴公司在杭州滨江区，有约 10 万名员工")
result = structured_llm.invoke("阿里巴巴公司在杭州滨江区，有约 10 万名员工")
result


提示: 阿里巴巴公司在杭州滨江区，有约 10 万名员工


Company(name='阿里巴巴', employee_count=100000, address=Address(city='杭州', district='滨江区'))

In [43]:
print(f"\n公司名称: {result.name}")
print(f"员工数量: {result.employee_count}")
print(f"地址: {result.address.city} - {result.address.district}")

print("\n关键点：")
print("  - Address 嵌套在 Company 中")
print("  - LLM 自动识别层级关系")
print("  - 通过 result.address.city 访问嵌套字段")



公司名称: 阿里巴巴
员工数量: 100000
地址: 杭州 - 滨江区

关键点：
  - Address 嵌套在 Company 中
  - LLM 自动识别层级关系
  - 通过 result.address.city 访问嵌套字段


In [44]:
print("""
    示例4：可选字段和默认值

    处理不完整的信息
    """)


    示例4：可选字段和默认值

    处理不完整的信息
    


In [45]:
print("\n" + "="*70)
print("示例 4：可选字段和默认值")
print("="*70)


示例 4：可选字段和默认值


In [46]:
class Product(BaseModel):
    """产品信息"""
    name: str = Field(description="产品名称")
    price: float = Field(description="价格")
    description: Optional[str] = Field(None, description="产品描述（可选）")
    stock: int = Field(100, description="库存（默认 100）")


In [47]:
structured_llm = create_safe_structured_llm(Product)
print("\n场景1：完整信息")
result1 = structured_llm.invoke("iPhone 15 售价 5999 元，最新款智能手机，库存 50 台")
print(f"  名称: {result1.name}")
print(f"  价格: {result1.price}")
print(f"  描述: {result1.description}")
print(f"  库存: {result1.stock}")


场景1：完整信息
  名称: iPhone 15
  价格: 5999.0
  描述: 最新款智能手机
  库存: 50


In [48]:
print("\n场景2：缺少描述和库存")
result2 = structured_llm.invoke("MacBook Pro 售价 12999 元")
print(f"  名称: {result2.name}")
print(f"  价格: {result2.price}")
print(f"  描述: {result2.description}")  # None
print(f"  库存: {result2.stock}")  # 100 (默认值)



场景2：缺少描述和库存
  名称: MacBook Pro
  价格: 12999.0
  描述: None
  库存: 100


In [49]:
print("\n关键点：")
print("  - Optional[str] 表示字段可以为 None")
print("  - Field(100, ...) 设置默认值")
print("  - LLM 未提供的信息会使用默认值")


关键点：
  - Optional[str] 表示字段可以为 None
  - Field(100, ...) 设置默认值
  - LLM 未提供的信息会使用默认值


In [50]:
class Priority(str, Enum):
    """优先级"""
    LOW = "低"
    MEDIUM = "中"
    HIGH = "高"

In [51]:
class Task(BaseModel):
    """任务"""
    title: str = Field(description="任务标题")
    priority: Priority = Field(description="优先级：低/中/高")
    completed: bool = Field(False, description="是否完成")


In [52]:
print("""
    示例5：枚举类型

    限制字段的可选值
    """)


    示例5：枚举类型

    限制字段的可选值
    


In [53]:
print("\n" + "="*70)
print("示例 5：枚举类型 - 限制可选值")
print("="*70)



示例 5：枚举类型 - 限制可选值


In [54]:
structured_llm = create_safe_structured_llm(Task)


In [55]:
print("\n提示: 完成季度报告，这是紧急任务")
result = structured_llm.invoke("完成季度报告，这是紧急任务")

print(f"\n任务: {result.title}")
print(f"优先级: {result.priority.value}")  # "高"
print(f"完成状态: {result.completed}")


提示: 完成季度报告，这是紧急任务

任务: 完成季度报告
优先级: 高
完成状态: False


In [56]:
print("\n关键点：")
print("  - Priority(str, Enum) 定义枚举")
print("  - LLM 只能选择 LOW/MEDIUM/HIGH")
print("  - 自动验证，无效值会报错")


关键点：
  - Priority(str, Enum) 定义枚举
  - LLM 只能选择 LOW/MEDIUM/HIGH
  - 自动验证，无效值会报错


In [57]:
class CustomerInfo(BaseModel):
    """客户信息"""
    name: str = Field(description="客户姓名")
    phone: str = Field(description="电话号码")
    email: Optional[str] = Field(None, description="邮箱（可选）")
    issue: str = Field(description="问题描述")
    urgency: Priority = Field(description="紧急程度")


In [58]:
"""
示例6：客户信息提取

从客服对话中提取结构化信息
"""
print("\n" + "="*70)
print("示例 6：实际应用 - 客户信息提取")
print("="*70)


示例 6：实际应用 - 客户信息提取


In [59]:
structured_llm = create_safe_structured_llm(CustomerInfo)

conversation = """
客服: 您好，请问有什么可以帮助您？
客户: 我是李明，电话 138-1234-5678，我的订单一直没发货，很着急！
客服: 好的，我帮您查一下
"""

In [60]:
print(f"\n对话记录:\n{conversation}")
result = structured_llm.invoke(f"从以下客服对话中提取客户信息：\n{conversation}")
result


对话记录:

客服: 您好，请问有什么可以帮助您？
客户: 我是李明，电话 138-1234-5678，我的订单一直没发货，很着急！
客服: 好的，我帮您查一下



CustomerInfo(name='李明', phone='138-1234-5678', email=None, issue='订单一直没发货', urgency=<Priority.HIGH: '高'>)

In [61]:
print("\n提取结果：")
print(f"  客户: {result.name}")
print(f"  电话: {result.phone}")
print(f"  邮箱: {result.email or '未提供'}")
print(f"  问题: {result.issue}")
print(f"  紧急程度: {result.urgency.value}")


提取结果：
  客户: 李明
  电话: 138-1234-5678
  邮箱: 未提供
  问题: 订单一直没发货
  紧急程度: 高


In [62]:
print("\n应用场景：")
print("  - 自动填充 CRM 系统")
print("  - 工单自动分类")
print("  - 紧急问题优先处理")


应用场景：
  - 自动填充 CRM 系统
  - 工单自动分类
  - 紧急问题优先处理


In [63]:
class Sentiment(str, Enum):
    """情感"""
    POSITIVE = "正面"
    NEUTRAL = "中性"
    NEGATIVE = "负面"

class Review(BaseModel):
    """评论"""
    product: str = Field(description="产品名称")
    rating: int = Field(description="评分 1-5")
    sentiment: Sentiment = Field(description="情感倾向")
    pros: List[str] = Field(description="优点列表")
    cons: List[str] = Field(description="缺点列表")

In [64]:
"""
示例7：产品评论分析

从自然语言评论中提取结构化数据
"""
print("\n" + "="*70)
print("示例 7：实际应用 - 产品评论分析")
print("="*70)


示例 7：实际应用 - 产品评论分析


In [65]:
structured_llm = create_safe_structured_llm(Review)

review_text = """
这款 iPhone 15 Pro 真的很不错！摄像头非常强大，夜拍效果惊艳。
钛金属边框手感也很好。但是价格有点贵，而且没有充电器。
总体来说还是值得购买的，我给 4 分。
"""

In [67]:
print(f"\n评论内容:\n{review_text}")
result = structured_llm.invoke(f"分析以下产品评论：\n{review_text}")
result


评论内容:

这款 iPhone 15 Pro 真的很不错！摄像头非常强大，夜拍效果惊艳。
钛金属边框手感也很好。但是价格有点贵，而且没有充电器。
总体来说还是值得购买的，我给 4 分。



Review(product='iPhone 15 Pro', rating=4, sentiment=<Sentiment.POSITIVE: '正面'>, pros=['摄像头强大', '夜拍效果惊艳', '钛金属边框手感好'], cons=['价格贵', '没有充电器'])

In [68]:
print("\n分析结果：")
print(f"  产品: {result.product}")
print(f"  评分: {result.rating} / 5")
print(f"  情感: {result.sentiment.value}")
print(f"  优点: {', '.join(result.pros)}")
print(f"  缺点: {', '.join(result.cons)}")


分析结果：
  产品: iPhone 15 Pro
  评分: 4 / 5
  情感: 正面
  优点: 摄像头强大, 夜拍效果惊艳, 钛金属边框手感好
  缺点: 价格贵, 没有充电器


In [69]:
print("\n应用场景：")
print("  - 批量处理用户评论")
print("  - 自动生成分析报告")
print("  - 发现产品改进点")


应用场景：
  - 批量处理用户评论
  - 自动生成分析报告
  - 发现产品改进点
